In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,accuracy_score
from xgboost import XGBClassifier
from xgboost import XGBRegressor

In [3]:
df_train = pd.read_csv("train_hp.csv")

In [4]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
df_train.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [6]:
df_train.shape

(1460, 81)

In [7]:
for col in [ 'Id', 'MSSubClass',   'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope',   'Condition1', 'Condition2', 'BldgType',
       'HouseStyle',   'OverallCond',   
       'RoofStyle', 'RoofMatl',   'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF',   
       'HeatingQC', 'CentralAir',     '2ndFlrSF',
       'LowQualFinSF',   'BsmtFullBath', 'BsmtHalfBath',  
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',  
           'Fireplaces',   
       'GarageYrBlt',    
           'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold',  
         ]:
    df_train.drop(col, axis=1, inplace = True)

In [8]:
df_train

,MSZoning,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,TotalBsmtSF,Heating,Electrical,1stFlrSF,...,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,SalePrice
0,RL,CollgCr,7,2003,2003,VinylSd,856,GasA,SBrkr,856,...,Attchd,RFn,2,548,TA,TA,Y,WD,Normal,208500
1,RL,Veenker,6,1976,1976,MetalSd,1262,GasA,SBrkr,1262,...,Attchd,RFn,2,460,TA,TA,Y,WD,Normal,181500
2,RL,CollgCr,7,2001,2002,VinylSd,920,GasA,SBrkr,920,...,Attchd,RFn,2,608,TA,TA,Y,WD,Normal,223500
3,RL,Crawfor,7,1915,1970,Wd Sdng,756,GasA,SBrkr,961,...,Detchd,Unf,3,642,TA,TA,Y,WD,Abnorml,140000
4,RL,NoRidge,8,2000,2000,VinylSd,1145,GasA,SBrkr,1145,...,Attchd,RFn,3,836,TA,TA,Y,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Gilbert,6,1999,2000,VinylSd,953,GasA,SBrkr,953,...,Attchd,RFn,2,460,TA,TA,Y,WD,Normal,175000
1456,RL,NWAmes,6,1978,1988,Plywood,1542,GasA,SBrkr,2073,...,Attchd,Unf,2,500,TA,TA,Y,WD,Normal,210000
1457,RL,Crawfor,7,1941,2006,CemntBd,1152,GasA,SBrkr,1188,...,Attchd,RFn,1,252,TA,TA,Y,WD,Normal,266500
1458,RL,NAmes,5,1950,1996,MetalSd,1078,GasA,FuseA,1078,...,Attchd,Unf,1,240,TA,TA,Y,WD,Normal,142125


In [9]:
numeric_columns=df_train.select_dtypes(include=[np.number])
print(numeric_columns.dtypes)

OverallQual     int64
YearBuilt       int64
YearRemodAdd    int64
TotalBsmtSF     int64
1stFlrSF        int64
GrLivArea       int64
FullBath        int64
TotRmsAbvGrd    int64
GarageCars      int64
GarageArea      int64
SalePrice       int64
dtype: object


In [10]:
corr=numeric_columns.corr()

In [11]:
print(corr['SalePrice'].sort_values(ascending=False))

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
Name: SalePrice, dtype: float64


In [12]:
df_train.head()

,MSZoning,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,TotalBsmtSF,Heating,Electrical,1stFlrSF,...,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,SalePrice
0,RL,CollgCr,7,2003,2003,VinylSd,856,GasA,SBrkr,856,...,Attchd,RFn,2,548,TA,TA,Y,WD,Normal,208500
1,RL,Veenker,6,1976,1976,MetalSd,1262,GasA,SBrkr,1262,...,Attchd,RFn,2,460,TA,TA,Y,WD,Normal,181500
2,RL,CollgCr,7,2001,2002,VinylSd,920,GasA,SBrkr,920,...,Attchd,RFn,2,608,TA,TA,Y,WD,Normal,223500
3,RL,Crawfor,7,1915,1970,Wd Sdng,756,GasA,SBrkr,961,...,Detchd,Unf,3,642,TA,TA,Y,WD,Abnorml,140000
4,RL,NoRidge,8,2000,2000,VinylSd,1145,GasA,SBrkr,1145,...,Attchd,RFn,3,836,TA,TA,Y,WD,Normal,250000


In [13]:
df_train.shape

(1460, 26)

In [14]:
df_train.isnull().any()

MSZoning         False
Neighborhood     False
OverallQual      False
YearBuilt        False
YearRemodAdd     False
Exterior1st      False
TotalBsmtSF      False
Heating          False
Electrical        True
1stFlrSF         False
GrLivArea        False
FullBath         False
KitchenQual      False
TotRmsAbvGrd     False
Functional       False
FireplaceQu       True
GarageType        True
GarageFinish      True
GarageCars       False
GarageArea       False
GarageQual        True
GarageCond        True
PavedDrive       False
SaleType         False
SaleCondition    False
SalePrice        False
dtype: bool

In [15]:
df_train.isnull().sum()

MSZoning           0
Neighborhood       0
OverallQual        0
YearBuilt          0
YearRemodAdd       0
Exterior1st        0
TotalBsmtSF        0
Heating            0
Electrical         1
1stFlrSF           0
GrLivArea          0
FullBath           0
KitchenQual        0
TotRmsAbvGrd       0
Functional         0
FireplaceQu      690
GarageType        81
GarageFinish      81
GarageCars         0
GarageArea         0
GarageQual        81
GarageCond        81
PavedDrive         0
SaleType           0
SaleCondition      0
SalePrice          0
dtype: int64

In [16]:
df_train['FireplaceQu'].isnull().sum()

690

In [17]:
df_train['FireplaceQu'] = df_train['FireplaceQu'].fillna(df_train['FireplaceQu'].mode()[0])

In [18]:
df_train['GarageType'] = df_train['GarageType'].fillna(df_train['GarageType'].mode()[0])
df_train['GarageFinish'] = df_train['GarageFinish'].fillna(df_train['GarageFinish'].mode()[0])
df_train['GarageQual'] = df_train['GarageQual'].fillna(df_train['GarageQual'].mode()[0])
df_train['GarageCond'] = df_train['GarageCond'].fillna(df_train['GarageCond'].mode()[0])
df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])

In [19]:
df_train.isnull().sum()

MSZoning         0
Neighborhood     0
OverallQual      0
YearBuilt        0
YearRemodAdd     0
Exterior1st      0
TotalBsmtSF      0
Heating          0
Electrical       0
1stFlrSF         0
GrLivArea        0
FullBath         0
KitchenQual      0
TotRmsAbvGrd     0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageCars       0
GarageArea       0
GarageQual       0
GarageCond       0
PavedDrive       0
SaleType         0
SaleCondition    0
SalePrice        0
dtype: int64

In [20]:
df_train

,MSZoning,Neighborhood,OverallQual,YearBuilt,YearRemodAdd,Exterior1st,TotalBsmtSF,Heating,Electrical,1stFlrSF,...,GarageType,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition,SalePrice
0,RL,CollgCr,7,2003,2003,VinylSd,856,GasA,SBrkr,856,...,Attchd,RFn,2,548,TA,TA,Y,WD,Normal,208500
1,RL,Veenker,6,1976,1976,MetalSd,1262,GasA,SBrkr,1262,...,Attchd,RFn,2,460,TA,TA,Y,WD,Normal,181500
2,RL,CollgCr,7,2001,2002,VinylSd,920,GasA,SBrkr,920,...,Attchd,RFn,2,608,TA,TA,Y,WD,Normal,223500
3,RL,Crawfor,7,1915,1970,Wd Sdng,756,GasA,SBrkr,961,...,Detchd,Unf,3,642,TA,TA,Y,WD,Abnorml,140000
4,RL,NoRidge,8,2000,2000,VinylSd,1145,GasA,SBrkr,1145,...,Attchd,RFn,3,836,TA,TA,Y,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Gilbert,6,1999,2000,VinylSd,953,GasA,SBrkr,953,...,Attchd,RFn,2,460,TA,TA,Y,WD,Normal,175000
1456,RL,NWAmes,6,1978,1988,Plywood,1542,GasA,SBrkr,2073,...,Attchd,Unf,2,500,TA,TA,Y,WD,Normal,210000
1457,RL,Crawfor,7,1941,2006,CemntBd,1152,GasA,SBrkr,1188,...,Attchd,RFn,1,252,TA,TA,Y,WD,Normal,266500
1458,RL,NAmes,5,1950,1996,MetalSd,1078,GasA,FuseA,1078,...,Attchd,Unf,1,240,TA,TA,Y,WD,Normal,142125


In [21]:
categorical=df_train.select_dtypes(exclude=[np.number]).dropna()
numerical=df_train.select_dtypes(include=[np.number]).dropna()

In [22]:
categorical

,MSZoning,Neighborhood,Exterior1st,Heating,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,CollgCr,VinylSd,GasA,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
1,RL,Veenker,MetalSd,GasA,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
2,RL,CollgCr,VinylSd,GasA,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
3,RL,Crawfor,Wd Sdng,GasA,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,WD,Abnorml
4,RL,NoRidge,VinylSd,GasA,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Gilbert,VinylSd,GasA,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,WD,Normal
1456,RL,NWAmes,Plywood,GasA,SBrkr,TA,Min1,TA,Attchd,Unf,TA,TA,Y,WD,Normal
1457,RL,Crawfor,CemntBd,GasA,SBrkr,Gd,Typ,Gd,Attchd,RFn,TA,TA,Y,WD,Normal
1458,RL,NAmes,MetalSd,GasA,FuseA,Gd,Typ,Gd,Attchd,Unf,TA,TA,Y,WD,Normal


In [23]:
numerical

,OverallQual,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,GarageArea,SalePrice
0,7,2003,2003,856,856,1710,2,8,2,548,208500
1,6,1976,1976,1262,1262,1262,2,6,2,460,181500
2,7,2001,2002,920,920,1786,2,6,2,608,223500
3,7,1915,1970,756,961,1717,1,7,3,642,140000
4,8,2000,2000,1145,1145,2198,2,9,3,836,250000
...,...,...,...,...,...,...,...,...,...,...,...
1455,6,1999,2000,953,953,1647,2,7,2,460,175000
1456,6,1978,1988,1542,2073,2073,2,7,2,500,210000
1457,7,1941,2006,1152,1188,2340,2,9,1,252,266500
1458,5,1950,1996,1078,1078,1078,1,5,1,240,142125


In [24]:
categorical_encoded = pd.get_dummies(categorical)

In [25]:
categorical_encoded

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1456,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1458,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0


In [26]:
categorical_encoded.shape

(1460, 109)

In [27]:
train_data = pd.concat([numerical, categorical_encoded], axis=1)
 

In [28]:
train_data

,OverallQual,YearBuilt,YearRemodAdd,TotalBsmtSF,1stFlrSF,GrLivArea,FullBath,TotRmsAbvGrd,GarageCars,GarageArea,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,7,2003,2003,856,856,1710,2,8,2,548,...,0,0,0,1,0,0,0,0,1,0
1,6,1976,1976,1262,1262,1262,2,6,2,460,...,0,0,0,1,0,0,0,0,1,0
2,7,2001,2002,920,920,1786,2,6,2,608,...,0,0,0,1,0,0,0,0,1,0
3,7,1915,1970,756,961,1717,1,7,3,642,...,0,0,0,1,1,0,0,0,0,0
4,8,2000,2000,1145,1145,2198,2,9,3,836,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,1999,2000,953,953,1647,2,7,2,460,...,0,0,0,1,0,0,0,0,1,0
1456,6,1978,1988,1542,2073,2073,2,7,2,500,...,0,0,0,1,0,0,0,0,1,0
1457,7,1941,2006,1152,1188,2340,2,9,1,252,...,0,0,0,1,0,0,0,0,1,0
1458,5,1950,1996,1078,1078,1078,1,5,1,240,...,0,0,0,1,0,0,0,0,1,0


In [136]:
 from sklearn import preprocessing
  
""" MIN MAX SCALER """
  
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1))
  
# Scaled feature
x_after_min_max_scaler = min_max_scaler.fit_transform(train_data)
  
print ("\nAfter min max Scaling : \n", x_after_min_max_scaler)
  
  
 


After min max Scaling : 
 [[0.66666667 0.94927536 0.88333333 ... 0.         1.         0.        ]
 [0.55555556 0.75362319 0.43333333 ... 0.         1.         0.        ]
 [0.66666667 0.93478261 0.86666667 ... 0.         1.         0.        ]
 ...
 [0.66666667 0.5        0.93333333 ... 0.         1.         0.        ]
 [0.44444444 0.56521739 0.76666667 ... 0.         1.         0.        ]
 [0.44444444 0.67391304 0.25       ... 0.         1.         0.        ]]


In [74]:
X=train_data.drop(['SalePrice'],axis=1)
y=train_data['SalePrice']

In [75]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=1)

In [76]:
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)
print(r2_score(y_test, y_pred))

0.7748693256263489


In [77]:
ligreg = LinearRegression()
ligreg.fit(X_train, y_train)
y_pred = ligreg.predict(X_test)
print(r2_score(y_test, y_pred))

0.8429601874740603


In [78]:
xg = XGBRegressor( booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=1.5, n_estimators=2400,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
xg.fit(X_train,y_train)
y_pred = xg.predict(X_test)
r2_score(y_test,y_pred)

0.9028972163591777